In [ ]:
import io
import json
from minio import Minio
from dotenv import dotenv_values

aws_credentials = dotenv_values("../env")

client = Minio(
    aws_credentials['MINIO_ENDPOINT_URL'],
    access_key=aws_credentials['ACCESS_KEY_ID'],
    secret_key=aws_credentials['SECRET_ACCESS_KEY'],
    secure=False
)

# Objects

In [ ]:
BUCKET = 'my-bucket'
OBJECT = 'output_1.json'

## [list_objects](https://min.io/docs/minio/linux/developers/python/API.html#list-objects-bucket-name-prefix-none-recursive-false-start-after-none-include-user-meta-false-include-version-false-use-api-v1-false-use-url-encoding-type-true)

Lists object information of a bucket.

In [ ]:
# List objects information.
objects = client.list_objects(BUCKET)
for obj in objects:
    print(obj.object_name)

In [ ]:
# List objects information whose names starts with "my/prefix/".
objects = client.list_objects(BUCKET, prefix="abc/")
for obj in objects:
    print(obj.object_name)

In [ ]:
# List objects information recursively.
objects = client.list_objects(BUCKET, recursive=True)
for obj in objects:
    print(obj.object_name)

In [ ]:
# List objects information recursively whose names starts with
# "my/prefix/".
objects = client.list_objects(
    BUCKET, prefix="abc", recursive=True,
)
for obj in objects:
    print(obj.object_name)

## [get_object](https://min.io/docs/minio/linux/developers/python/API.html#get-object-bucket-name-object-name-offset-0-length-0-request-headers-none-ssec-none-version-id-none-extra-query-params-none)

Gets data from offset to length of an object. Returned response should be closed after use to release network resources. To reuse the connection, it’s required to call response.release_conn() explicitly.

In [ ]:
response = client.get_object(BUCKET, OBJECT)

In [ ]:
response.data.decode()

## [put_object](https://min.io/docs/minio/linux/developers/python/API.html#put_object)

Uploads data from a stream to an object in a bucket.

In [ ]:
json_file = {"clientes": [{"id": "1b484d189cafc68008fbcb1adef94082", "name": "BARTOLOMEU", "idade": 38, "credito_solicitado": 230311, "data_solicitacao": "01/15/2022 08:11"}, {"id": "c4e0c38c502b109a9324cf8fd90ef907", "name": "NIGEL", "idade": 54, "credito_solicitado": 345304, "data_solicitacao": "01/15/2022 16:48"}, {"id": "89d21736d351f3a6445c91f753b1fb1f", "name": "MAXIMILLIAN", "idade": 60, "credito_solicitado": 423406, "data_solicitacao": "01/23/2022 18:10"}, {"id": "48d605f6dfa428020ba67298ad1a4041", "name": "AHIL", "idade": 54, "credito_solicitado": 402914, "data_solicitacao": "01/02/2022 03:56"}, {"id": "5e0f2688c5f1c64c92df687ac4917ce3", "name": "INDIE", "idade": 64, "credito_solicitado": 182364, "data_solicitacao": "01/19/2022 08:16"}]}

json_record = json.dumps(json_file)

In [ ]:
result = client.put_object(
    bucket_name=BUCKET, 
    object_name=OBJECT, 
    data=io.BytesIO(json_record.encode()), 
    length=-1, 
    part_size=10000000
)

# Remove

## [remove_object](https://min.io/docs/minio/linux/developers/python/API.html#remove-object-bucket-name-object-name-version-id-none)

Remove an object.

In [ ]:
client.remove_object(BUCKET, OBJECT)

## [remove_objects](https://min.io/docs/minio/linux/developers/python/API.html#remove-objects-bucket-name-delete-object-list-bypass-governance-mode-false)

Remove multiple objects.

In [ ]:
from minio.deleteobjects import DeleteObject

In [ ]:
client.remove_objects(
    BUCKET,
    [
        DeleteObject("output_7.json"),
        DeleteObject("output_8.json")
    ],
)

# Copy

## [copy_object](https://min.io/docs/minio/linux/developers/python/API.html#copy-object-bucket-name-object-name-source-sse-none-metadata-none-tags-none-retention-none-legal-hold-false-metadata-directive-none-tagging-directive-none)

Create an object by server-side copying data from another object. In this API maximum supported source object size is 5GiB.

In [ ]:
from minio.commonconfig import CopySource

In [ ]:
SOURCE_BUCKET = 'my-bucket'
SINK_BUCKET = 'velero'
OBJECT = 'output_1.json'

In [ ]:
client.copy_object(
    SINK_BUCKET,
    OBJECT,
    CopySource(SOURCE_BUCKET, OBJECT),
)